# 日本語(Japanese)EDAで、個人的な解釈です。
## memo書きみたいなものもあるので、言い回しが丁寧ではありません・・・



# 0. inputファイルの確認 (confirming input files)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

import matplotlib.pyplot as plt

# 1. train.csvの確認(confirming train.csv)
## ※ 1億行あるので、100万行でまずは読む (I read only 1 million rows because this file is composed of over 10 million.

In [ ]:
traindf = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv",nrows=10**6 )
traindf

In [ ]:
traindf.info()

♯ メモリーを効率的に使うための型変換 (他コンペから抜粋)

In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
traindf = reduce_mem_usage(traindf)

In [ ]:
traindf.info()

In [ ]:
traindf.head(3)

 # <span style="background-color: yellow; ">1.train.csv
## siteに細かい説明あるが、わかりやすく解釈したもの (Below sentence is my interpretation in order to understand easily.

1.  row_id : dataframeのid
2.  timestamp : 生徒の直前のアクションからの経過時間(ミリ秒)
3.  user id : 生徒のid
4.  content_id : 問題のid
5.  content_type_id : 質問(0)か講義(1)
6.  task_container_id : 
 わかりやすく捉えると、4.content idの章みたいな感じ (TOEICの問題をイメージすると、１つの長文がこれに相当して、それの問題1,2,3みたいのが、content idでそれぞれ振り分けられている)
6.  user_answer : 4択 (0,1,2,3)でどれを答えたか
7.  answered_correctly : それがあっていたかどうか
8.  prior_question_elapsed_time	 : 1つ前の質問(3個質問がある場合がある)からの経過時間
9.  prior_question_had_explanation : 1つ前の質問(3個質問がある場合がある)の後に、解説および回答を見る機会があったか

## 最終的には、answered_correctly以外の情報から、問題を当てられるかどうかを0～1の確率で出すコンペティション
## (testデータにはもう少し細工が入っている)


In [ ]:
for a in traindf.columns:
    print(a + ":" + str(len(traindf[a].unique())))

# 100万個データから以下のことがわかる
* 生徒(user_id)が3824人
* 問題(content_id)が13320個
* 問題 or 講義で２個(content_type_id)
* 問題の章(task_container_id)が7740個
* 生徒の回答の選択肢が５個 (-1,0,1,2,3)で-1は講義
![](http://)* 正解 or not で３個(-1,0,1) で-1は講義

#### わかりやすく理解するために、user id:115のみを見てみる

In [ ]:
traindf115 = traindf[traindf["user_id"]==115]
traindf115

#### task container idは、1～41章あって、40と41はそれぞれ問題が3個ある
#### prior_question_elapsed_timeは講義(問題後の解説とか)を抜かしたタイムのため、単純に引き算ではなさそう。

In [ ]:
# timestampの調査
px.line(traindf115,x="row_id",y="timestamp",width = 500)

#### row idが38→39問目になったときに急激に上がっている。TOEICで言うとlisteningとreadingで間が空くとかそんなイメージ ?
#### ※　後ほどquestion.csvとマージするとpartが変わっている

In [ ]:
# 正答率
traindf115["answered_correctly"].sum()/len(traindf115)

# 100万個のデータから生徒ごとの正答率を出してみる。

In [ ]:
traindf

In [ ]:
traindf.describe()

In [ ]:
# answered_correctlyにmin -1 が入っている　→　質問と講義が入っているため

In [ ]:
# 質問と講義に分ける

In [ ]:
qdf = traindf[traindf["content_type_id"]==0]
qdf

In [ ]:
qdf.describe()

#### 分離できた

#### 生徒ごとの正答率は、愚直にやれば以下の感じ・・・でも、時間かかると思われる

In [ ]:
userlist = qdf["user_id"].unique()
userlist

# 0だけをまずやる

In [ ]:
userlist[0]

In [ ]:
tmpdf = qdf[qdf["user_id"]==userlist[0]]
tmpdf.head(3)

In [ ]:
tmpdf["answered_correctly"].sum()/len(tmpdf)

# まとめてfor文で回す

In [ ]:
ratelist= []
for a in userlist:
    tmpdf = qdf[qdf["user_id"]==a]
    ratelist.append([a,tmpdf["answered_correctly"].sum()/len(tmpdf)])

In [ ]:
ratelistdf = pd.DataFrame(ratelist)
ratelistdf.columns = ["user_id","rate"]
ratelistdf

# 玄人はgroupbyで一発でやる

In [ ]:
userdf = qdf.groupby('user_id')
userdf

In [ ]:
userdf_calc = userdf.agg({'answered_correctly': ['mean',"sum", 'count', 'std']}).copy() # .aggでその中身をmeanやsumなど複数処理可能。
userdf_calc.columns = [
    'mean', 
    "sum",
    'count', 
    'std', 
 ]

userdf_calc

In [ ]:
userdf_calc["rate"] = userdf_calc["sum"]/userdf_calc["count"]

In [ ]:
userdf_calc

In [ ]:
px.histogram(
    userdf_calc, 
    x="rate",
    nbins=50,
    width = 500
)



#### 生徒ごとに、約60%くらいの正答率であることがわかる

# 関数化　↑のをコピペして作成しているので、userdfとなっているが気にしない。

In [ ]:
def makingrate(col):
    

    userdf = qdf.groupby(col)
    userdf_calc = userdf.agg({'answered_correctly': ['mean',"sum", 'count', 'std']}).copy() # .aggでその中身をmeanやsumなど複数処理可能。
    userdf_calc.columns = [
        'mean', 
        "sum",
        'count', 
        'std', 
     ]
    userdf_calc["rate"] = userdf_calc["sum"]/userdf_calc["count"]
    fig = px.histogram(
        userdf_calc, 
        x="rate",
        nbins=50,
        width = 500
    )
    fig.show()
    return userdf_calc


In [ ]:
qdf

In [ ]:
cdf = makingrate("content_id")
cdf

#### 問題ごとの正答率が完成。
#### 0 →　むちゃくちゃ難しい問題 ?, 1→むちゃくちゃ簡単な問題 ? 、その他はきれいなヒストグラム　←　train.csvを全部読むときれいになります。0と1が多くなるとかがなくなります。

In [ ]:
tdf = makingrate("task_container_id")
tdf

#### 章ごとの正答率が完成。
#### countが１個しかない章とかが正答率1とかになっているのか。　←　train.csvを全部読むときれいになります。0と1が多くなるとかがなくなります。

In [ ]:
tdf[tdf["count"]==1]

# 上記の0,1は1個しか問題がないケースが多そう。2040個もある

In [ ]:
px.line(tdf,x=tdf.index ,y="count",width=500)

# 章はidが大きくなると数が少なくなっている

In [ ]:
px.line(tdf,x=tdf.index ,y="rate",width=500)

#### 上記2つから、task_contaier_idが小さくなるほど正答率が収束している。1億個のビッグデータだとどうなるのか・・・

In [ ]:
qdf

In [ ]:
ptime = makingrate("prior_question_elapsed_time")
ptime

In [ ]:
px.line(ptime,x=ptime.index,y="count")

In [ ]:
px.line(ptime,x=ptime.index,y="rate")

In [ ]:
explanation = makingrate("prior_question_had_explanation")
explanation

#### 問題の前に解説などあると、正解率が高い

# <span style="background-color: yellow; ">2.question.csv

In [ ]:
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
questions.head()

* ** question_id **：traincsvのcontent id

* ** bundle_id **：どの質問と同時に送られたか ?

* ** correct_answer **：問題の答え

* **パート**：TOEICテストの関連セクション。

* **タグ**：質問の1つ以上の詳細なタグコード。タグの意味は提供されませんが、これらのコードは質問をまとめるのに十分です。

In [ ]:
questions["tags"] = questions["tags"].astype("str")
tags = questions["tags"].apply(lambda x: pd.Series(x.split())) # googleでdataframe　スペース区切りとかやると出てくる

In [ ]:
tags

In [ ]:
tags.columns = ["tag0","tag1","tag2","tag3","tag4","tag5"]
tags

In [ ]:
questions = pd.concat([questions,tags],axis=1)
questions = questions.drop("tags",axis=1)
questions

In [ ]:
questions.describe()

In [ ]:
qdf.describe()

# content idとquestion idが同じなので、mergeしてしまう

In [ ]:
questions.columns

In [ ]:
questions.columns=['content_id', 'bundle_id', 'correct_answer', 'part', 'tag0', 'tag1',
       'tag2', 'tag3', 'tag4', 'tag5']
questions

In [ ]:
qdf2 = pd.merge(qdf,questions,on="content_id",how="left")
qdf2

In [ ]:
qdf115 = qdf2[qdf2["user_id"]==115]

In [ ]:
qdf115.tail(30)

#### bundle idは３つ質問があったら、それと紐づいている ex 3363-3365の問題は3363と一緒にされたもの　という意味。
#### partの1,2,3,4で時間ロスが大きく変わっているので、問題の傾向が変わっている
#### tagは問題の形式かな ? 何かが入っていると正答率が悪いとかあれば、面白い　meltしちゃって、並べても良いが、1億行にさらに増やしてもな・・・と迷いどころ
#### ×6の6億個になってしまう

# partごとの正答率を出してみる

In [ ]:
# さきほどのmakingrate2のqdfをqdf2に変えただけ
def makingrate2(col):
    

    userdf = qdf2.groupby(col)
    userdf_calc = userdf.agg({'answered_correctly': ['mean',"sum", 'count', 'std']}).copy() # .aggでその中身をmeanやsumなど複数処理可能。
    userdf_calc.columns = [
        'mean', 
        "sum",
        'count', 
        'std', 
     ]
    userdf_calc["rate"] = userdf_calc["sum"]/userdf_calc["count"]
    
   
    return userdf_calc


In [ ]:
partdf = makingrate2("part")
partdf

In [ ]:
px.bar(partdf,x=partdf.index,y="rate",width=500)

## tagごとの正答率を見る

In [ ]:
qdf2

In [ ]:
qdf2_tag = qdf2[["row_id","content_id","answered_correctly","tag0","tag1","tag2","tag3","tag4","tag5"]]

In [ ]:
qdf2_tag

In [ ]:
qdf_melt = pd.melt(qdf2_tag,id_vars=["row_id","content_id","answered_correctly"],var_name="tag",value_name="tagvalue")

In [ ]:
qdf_melt

# nanデータの削除の練習

In [ ]:
qdf_melt["tagvalue"].unique()

# nan という文字列がある。これとNaNが別物

In [ ]:
qdf_melt["tagvalue"] =  pd.to_numeric( qdf_melt['tagvalue'], errors='coerce') # nanという文字列をNaNに変えているのだと思います。

In [ ]:
qdf_melt = qdf_melt.dropna(how="any")
qdf_melt

In [ ]:
# さきほどのmakingrate2のqdfをqdf2に変えただけ
def makingrate3(col):
    

    userdf = qdf_melt.groupby(col)
    userdf_calc = userdf.agg({'answered_correctly': ['mean',"sum", 'count', 'std']}).copy() # .aggでその中身をmeanやsumなど複数処理可能。
    userdf_calc.columns = [
        'mean', 
        "sum",
        'count', 
        'std', 
     ]
    userdf_calc["rate"] = userdf_calc["sum"]/userdf_calc["count"]
    
   
    return userdf_calc


In [ ]:
tagdf = makingrate3("tagvalue")
tagdf

In [ ]:
px.bar(tagdf,x=tagdf.index,y="mean")

# tagの順番にも何か影響がありそう。

In [ ]:
tagdf2 = makingrate3("tag")
tagdf2

In [ ]:
px.bar(tagdf2,x=tagdf2.index,y="mean",width=500)

In [ ]:
px.line(tagdf2,x=tagdf2.index,y="count",width=500)

# 後ろのほうにかかれているtagのほうが正答率高いかも。データ少ないだけ ?

# <span style="background-color: yellow; ">3.lecture.csv

In [ ]:
## traindfの中でもlectureだけを抜く

In [ ]:
traindf

In [ ]:
ldf = traindf[traindf["content_type_id"]==1]
ldf

In [ ]:
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
lectures

* lecture id : traindfのcontent id
* tag : 講義用の1つのタグコード。タグの意味は提供されませんが、これらのコードは講義をまとめるのに十分です。
* part : 講義のトップレベルのカテゴリコード。
* type of : 講義のタイプ


In [ ]:
# これもマージしてみる

In [ ]:
lectures.columns

In [ ]:
lectures.columns = ['content_id', 'tag', 'part', 'type_of']

In [ ]:
ldf2 = pd.merge(ldf,lectures,on="content_id",how="left")
ldf2

In [ ]:
ldf2[ldf2["user_id"]==20938253]

# 講義は意味あるのだろうか・・・

In [ ]:
tdf20938253 = traindf[traindf["user_id"]==20938253]
tdf20938253

In [ ]:
tdf20938253["content_type_id"].plot()

# 講義見て、問題解いて・・・とかのケースがあるということだと推測。

# いったんまとめ
## -- 正答率に影響ありそうなパラメータ --
* content_id : 問題
* prior_question_had_explanation : 前の問題のあと説明や解説があったか。
* part : TOEICのパート。リスニング、リーディングなど
* task_contaier_id : 章
* tag : 問題のタグ
* tag no : 問題のタグが最初から何番目にかかれているか

## --- まだわからないところ ---
* lectureを受けた後、正答率が上がるか ?
* user idはtestデータでも重複する可能性があるか ?

# <span style="background-color: yellow; ">4. example_test.csv

In [ ]:
test_ex = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
test_ex

In [ ]:
traindf.head(3)

## train.csvと違うところ
* group numberがある
* prior_groups_answers_correctがある
* priot_groups_responsesがある
* answered_correctlyがない。（これを推測)

In [ ]:
test_ex

In [ ]:
test_ex[test_ex["user_id"]==275030867]

In [ ]:
test_ex[test_ex["user_id"]==554169193]

In [ ]:
g0 = test_ex[test_ex["group_num"]==0]
g0

In [ ]:
g1 = test_ex[test_ex["group_num"]==1]
g1

#### 0行目に前のgroupが正解したかなどの情報がはいっている

In [ ]:
tmp = g1.iloc[0,-2]
tmp

In [ ]:
tmp = g1.iloc[0,-2]
tmp = tmp.split("[")[1]
tmp = tmp.split("]")[0]
tmp = tmp.split(",")
tmp

In [ ]:
def mojihenkan(mojilist):
    tmp = mojilist
    tmp = tmp.split("[")[1]
    tmp = tmp.split("]")[0]
    tmp = tmp.split(",")
    return tmp

In [ ]:
g0["correct_ornot"] = mojihenkan(g1.iloc[0,-2])
g0["choice"] = mojihenkan(g1.iloc[0,-1])

In [ ]:
g0

# group(時間?)が進むと、前の回答があっていたかどうかの判定が出る。
# 提出方法は ?

# <span style="background-color: yellow; ">5. about submission

In [ ]:
# このコンペティションの提出方法は特殊。以下、スターターnotebookから、解釈。
# inputの中にriiideducationというpyファイルが入っているので、それをimport

In [ ]:
import riiideducation
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()

In [ ]:
(test_df, sample_prediction_df) = next(iter_test) # 中身確認のみ本番は消す。


In [ ]:
test_df

In [ ]:
sample_prediction_df

#### Note that we'll get an error if we try to continue on to the next batch without making our predictions for the current batch.
#### 現在のバッチの予測を作ることなしに次のバッチに行こうとするとエラーが出るよ。と書いてあるので、
#### 1 groupずつ、predictionして、保存して、次に行くということをやるのだと推測。

In [ ]:
# 保存方法 # 中身確認のみ本番は消す。
env.predict(sample_prediction_df)

In [ ]:
(test_df, sample_prediction_df) = next(iter_test) # 中身確認のみ本番は消す。

In [ ]:
test_df

In [ ]:
env.predict(sample_prediction_df)

# 注意点 :
# 講義のところは提出しないとある
# traincsvの中にない問題(content_id)があるとも記述あり
# ですが、訂正されて、traincsvの中にない問題(content_id)はないそうです(コメントありがとうございます)

* ---大本の記述---
* https://www.kaggle.com/sohier/competition-api-detailed-introduction　にはそのように書いてありますが、

* ---訂正のお知らせサイト---
* https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/191106

* ただし、このnotebookの場合、100万行のtrainデータなので、足りません・・・。
* 参考 100万行のcontent_id数 : 13320個。　trainデータ全部(約1億行)のcontent_id数 : 13782個

In [ ]:
"""
basic codeは以下だが、つまらないので、少し変更する

for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = 0.5
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

"""



In [ ]:
test_df

In [ ]:
cdf=cdf.reset_index()
cdf

In [ ]:
cdf.rate.mean()

In [ ]:
test_df2 = pd.merge(test_df,cdf,on="content_id",how="left")
test_df2

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[test_df["content_type_id"]==0] # 質問only
    test_df = pd.merge(test_df,cdf,on="content_id",how="left") # 問題ごとの正答率をマージ
    test_df['answered_correctly'] = test_df["rate"]
    test_df = test_df[["content_type_id","row_id","answered_correctly"]] # この後のfillnaをするときに、他の列があると、型が違くて無理というエラーが出ることがあるため
    
    test_df.fillna(value = 0.696, inplace = True) # 今回のnotebook(100万行データ)でない問題は、全部の問題の正答率の平均を使う。
    
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])


# 注意点 : 
1. internet = offでcommitしないと登録できないので、注意。
1. make_env() が１回しかできないので、最初のグループと2個目のグループはこのnotebookだとtest_dfを解説用に表示したが、このままだと最初と2個目は、0.5に全部なるので、修正必要
1. 100万個のデータであることに注意。全部train.csvを読み込んで(約1億行)、同様に問題ごとの平均値を入れた時のスコアは0.703(別のnotebookで試しました)